In [43]:
# Librerias
import random

In [44]:
# -------------------------- Calcular puntaje -----------------------
# Función para eliminar columnas de gaps
def eliminar_columnas_con_gaps(alineamiento):
    columnas_para_eliminar = []
    longitud_maxima = max(len(secuencia) for secuencia in alineamiento) # Num. maximo de caracteres en todas las secuencias.

    # Identifica las columnas que solo contienen gaps
    for i in range(longitud_maxima):  # Hasta la longitud máxima
        # Comprueba si todas las secuencias tienen un guion en la posición i
        columna_gap = all(
            i < len(alineamiento[j]) and alineamiento[j][i] == '-' for j in range(len(alineamiento))
        )
        if not columna_gap:
            columnas_para_eliminar.append(i)

    # Crear un nuevo alineamiento sin esas columnas
    nuevo_alineamiento = []
    for secuencia in alineamiento:
        nueva_secuencia = "".join(
            [secuencia[i] for i in columnas_para_eliminar if i < len(secuencia)]
        )
        nuevo_alineamiento.append(nueva_secuencia)

    return nuevo_alineamiento

# Función para calcular el puntaje de un alineamiento CORREGIR
def CalcularPuntaje(alineamiento, contador_secuencia):
    alineamiento_sin_gaps = eliminar_columnas_con_gaps(alineamiento)  # Elimina las columnas de gaps
    puntaje = 0
    longitud_sin_gaps = min(len(secuencia) for secuencia in alineamiento_sin_gaps)
    for i in range(longitud_sin_gaps):  # Solo esta puntuando hasta el minimo para evitar errores CORREGIR
        coincidencias = sum(
            alineamiento_sin_gaps[j][i] == alineamiento_sin_gaps[0][i] for j in range(contador_secuencia)
        )
        puntaje += coincidencias / contador_secuencia
    return puntaje
#--------------------------------------------------------------------

# ---------------------- Generar poblacion --------------------------
# Función para insertar un carácter en una posición específica
def insertar_caracter(cadena, caracter, posicion):
    return cadena[:posicion] + caracter + cadena[posicion:]

# Función para generar alineamientos aleatorios
def generar_random_alineamiento(secuencias):
    alineamiento = [secuencia for secuencia in secuencias]  # Clonar las secuencias originales
    for i in range(len(secuencias[0])):
        for j in range(len(secuencias)):
            if random.random() < 0.5:  # Decidir si se inserta un gap
                # Insertar un guion en la posición j en el índice actual (i)
                posicion_insercion = random.randint(0, i)  # Elegir una posición para insertar el guion
                alineamiento[j] = insertar_caracter(alineamiento[j], '-', posicion_insercion)
            # Siempre añadir el carácter actual de la secuencia
            if len(secuencias[j]) == i:
                alineamiento[j] += secuencias[j][i]
    return alineamiento
# --------------------------------------------------------------------

# --------------------------------- Cruza y mutacion de hijos ------------------------------------
# Función para realizar el cruzamiento entre dos alineamientos
def Cruza(padre1, padre2):
    punto_cruza = random.randint(0, len(padre1[0]))
    hijo1 = [padre1[j][:punto_cruza] + padre2[j][punto_cruza:] for j in range(len(padre1))]
    hijo2 = [padre2[j][:punto_cruza] + padre1[j][punto_cruza:] for j in range(len(padre2))]
    return hijo1, hijo2

# Función para aplicar mutación a un alineamiento
def mutacion(alineamiento, valor_mutacion):
    mutacion_alineamiento = ['' for _ in range(len(alineamiento))]  # Alineamiento vacío
    longitud_maxima = max(len(secuencia) for secuencia in alineamiento) # Longitud maxima
    for i in range(longitud_maxima):  
        for j in range(len(alineamiento)):                
            if i < len(alineamiento[j]) and random.random() < valor_mutacion:
                posicion_insercion = random.randint(0, i)  # Posición aleatoria
                mutacion_alineamiento[j] = insertar_caracter(mutacion_alineamiento[j], '-', posicion_insercion)
                mutacion_alineamiento[j] += alineamiento[j][i]
    return mutacion_alineamiento
# ----------------------------------------------------------------------------------------------

In [45]:
# Parámetros de entrada
secuencias = ["MURCIELAGO--", "ENUMERADO---","ESTRUENDO---","MUERDAGO----"]
tamaño_poblacion = 20
valor_mutacion = 0.1
generaciones = 100

# Creacion de la población
poblacion = [generar_random_alineamiento(secuencias) for _ in range(tamaño_poblacion)]

# Ciclo principal
for generation in range(generaciones):
    # Puntaje de aptitud de cada alineamiento
    coincidencia_puntajes = [CalcularPuntaje(alineamiento, len(secuencias)) for alineamiento in poblacion]
    
    # Selección de padres
    padres = random.choices(poblacion, weights=coincidencia_puntajes, k=2)
    
    # Cruzamiento
    hijo1, hijo2 = Cruza(padres[0], padres[1])
    
    # Mutación
    hijo1 = mutacion(hijo1, valor_mutacion)
    hijo2 = mutacion(hijo2, valor_mutacion)
    
    # Reemplazar los alineamientos
    poblacion[coincidencia_puntajes.index(min(coincidencia_puntajes))] = hijo1
    poblacion[coincidencia_puntajes.index(min(coincidencia_puntajes))] = hijo2

# Seleccionar el mejor alineamiento de la última generación
mejor_alineamiento = max(poblacion, key=lambda x: CalcularPuntaje(x, len(secuencias)))
print("Mejor alineamiento:")
for i, seq in enumerate(secuencias):
    print("Secuencia", i+1, ":", mejor_alineamiento[i])
print("Puntaje de aptitud:", CalcularPuntaje(mejor_alineamiento, len(secuencias)))


Mejor alineamiento:
Secuencia 1 : ---M--U-RCIELAGO--
Secuencia 2 : ---EN-U--M-ERADO---
Secuencia 3 : ------ESTRUENDO---
Secuencia 4 : -MU-ERDA-G-O----
Puntaje de aptitud: 7.0
